# Pytorch Training Tutorial

In [17]:
import timm
import torch
import numpy as np
import scipy.io as sio
import torch.nn.functional as F
import torch.nn
import time
from tqdm import tqdm
from torchvision import datasets
from torchvision import transforms as T 
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

import torchsummary

import copy

from timm.data import Mixup
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [18]:
print(torch.__version__)

1.13.1


In [19]:
config={    
    'data_test_dir': r'D:\Coding\dataset\gender_kaggle\Test',
    'data_train_dir': r'D:\Coding\dataset\gender_kaggle\Train',
    'data_single_dir': r'D:\Coding\dataset\flowers',
    
    'batch_size':64,
    'num_classes':2 ,
    'epochs': 300
    }

In [20]:
'''
Torch Transform -> Data Augmentation
'''
transform = T.Compose([
    #T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    # T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
    # T.RandomAffine(degrees=15, translate=(0.2, 0.2),scale=(0.8, 1.2), shear=15),
    # T.RandomHorizontalFlip(),
    # T.RandomVerticalFlip(),
    # T.RandomRotation(10),
    T.RandAugment(num_ops = 9, magnitude = 3),
    T.Resize((224,224)),
    #T.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
    T.RandomErasing(p=0.25, value='random')
    ])

In [21]:
'''
ImageFolder : 실제로 불러오는 것은 파일별 경로 및 레이블
Dataset Shape -> dataset[data_index][0]:label, dataset[data_index][1]:image
테스트 데이터셋과 트레이닝 데이터셋을 따로 불러옴
'''
train_ds = datasets.ImageFolder(root=config['data_train_dir'], transform =transform)
test_ds = datasets.ImageFolder(root=config['data_test_dir'], transform =T.Compose([T.Resize((224,224)),T.ToTensor(),T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) ]))

In [22]:
'''
# 트레이닝 데이터셋만 있을 경우 ImageFolder에서 train_ds만 불러온 후
# Subsampler을 활용하여 검증 데이터셋과 트레이닝 데이터셋, 테스트 데이터셋으로 나눔

def get_sampler(data_len, val):
    # data_len 은 트레이닝 데이터셋의 길이
    valid_size = 0.1
    data_len = len(train_ds)


    indices = list(range(data_len)) # 전체 인덱스
    np.random.shuffle(indices)

    split = int(np.floor(valid_size * data_len))
    train_idx, valid_idx,test_idx = indices[:-split*2], indices[-split*2:-split], indices[-split:]

    # trainning, validation batch를 얻기 위한 sampler정의
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    train_len = len(train_idx)
    val_len = len(valid_idx)
    test_len = len(test_idx)

    return train_sampler, valid_sampler, test_sampler, train_len, val_len, test_len

train_ds = datasets.ImageFolder(root=config['data_single_dir'], transform =transform)
train_idx, valid_idx, test_idx, train_len, val_len, test_len = get_sampler(data_len=len(train_ds), val=0.1)
'''

"\n# 트레이닝 데이터셋만 있을 경우 ImageFolder에서 train_ds만 불러온 후\n# Subsampler을 활용하여 검증 데이터셋과 트레이닝 데이터셋, 테스트 데이터셋으로 나눔\n\ndef get_sampler(data_len, val):\n    # data_len 은 트레이닝 데이터셋의 길이\n    valid_size = 0.1\n    data_len = len(train_ds)\n\n\n    indices = list(range(data_len)) # 전체 인덱스\n    np.random.shuffle(indices)\n\n    split = int(np.floor(valid_size * data_len))\n    train_idx, valid_idx,test_idx = indices[:-split*2], indices[-split*2:-split], indices[-split:]\n\n    # trainning, validation batch를 얻기 위한 sampler정의\n    train_sampler = SubsetRandomSampler(train_idx)\n    valid_sampler = SubsetRandomSampler(valid_idx)\n    test_sampler = SubsetRandomSampler(test_idx)\n\n    train_len = len(train_idx)\n    val_len = len(valid_idx)\n    test_len = len(test_idx)\n\n    return train_sampler, valid_sampler, test_sampler, train_len, val_len, test_len\n\ntrain_ds = datasets.ImageFolder(root=config['data_single_dir'], transform =transform)\ntrain_idx, valid_idx, test_idx, train_len, val_len, test_len =

In [23]:
'''
train_loader = DataLoader(train_idx, batch_size=config['batch_size'],sampler=None ,shuffle=True, num_workers=0)
val_loader = DataLoader(valid_idx, batch_size=config['batch_size'],sampler=None,shuffle=True, num_workers=0)
'''

"\ntrain_loader = DataLoader(train_idx, batch_size=config['batch_size'],sampler=None ,shuffle=True, num_workers=0)\nval_loader = DataLoader(valid_idx, batch_size=config['batch_size'],sampler=None,shuffle=True, num_workers=0)\n"

In [24]:
'''
DataLoader : 데이터로더는 데이터셋을 배치 단위로 불러올 수 있게 해주는 역할 -> 경로를 기반으로 실제로 데이터를 배치단위로 불러옴

sampler -> 샘플 인덱스 지정해주는 역할 (이전의 SubsetRandomSampler의 결과가 인풋)
shuffle -> 셔플링 여부 
둘중에 하나만 쓰는 것이 좋음
num_workers -> 멀티프로세싱을 몇개로 해줄지: 0이면 싱글프로세싱, 주로 CPU 개수 만큼 설정합니다
'''
train_loader = DataLoader(train_ds, batch_size=config['batch_size'],sampler=None ,shuffle=True, num_workers=2)
val_loader = DataLoader(test_ds, batch_size=config['batch_size'],sampler=None,shuffle=True, num_workers=2)

In [25]:
# 허깅페이스 사전학습 모델 로딩
model = timm.create_model('vit_small_patch16_224', pretrained=True, num_classes=100)

In [26]:
# 가장 간단한 형태의 병렬화
#model =  torch.nn.DataParallel(model,device_ids=[0,1])

'''
모델 병렬학습 코드
1. torch.nn.DataParallel : 

장점: 정말 간단

단점 : DataParallel은 쓰레드간 GIL 경합, 복제 모델의 반복 당 생성, 산란 입력 및 수집 출력으로 인한 추가적인 오버헤드로 인해 
단일 시스템에서도 istributedDataParallel보다 느립니다. 또한 

2. torch.nn.DistributedDataParallel : 다중 작업이며 단일 및 다중 기기 학습을 전부 지원

장점: 빠르다
단점 : 추가적인 함수 작성 필요
-> 여러 개의 GPU에서 딥러닝을 실행하려면, 모델을 복사해서 각  GPU에 할당해야한다. 
-> 그 뒤 batchsize를 batch_size/num_gpu만큼 나눈다. 이것을 scatter 한다고 표현한다. (실제로 scatter 함수가 있다.)

각 GPU에서 모델이 입력을 받아 출력하는 것을 forward 한다고 표현하고, 이 출력들을 하나의 GPU로 모은다. 이렇게 여러 tensor들(출력들)을 하나의 device로 모으는 것을 gather라고 한다.
'''

# model 을 GPU로 올림 model.to('cuda')와 동일
model.cuda()

# 모델 요약 정보
# DataParallel을 사용하면 요약 시 모델사이즈가 GPU 병렬사용 갯수만큼 커짐
torchsummary.summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 384, 14, 14]         295,296
          Identity-2             [-1, 196, 384]               0
        PatchEmbed-3             [-1, 196, 384]               0
           Dropout-4             [-1, 197, 384]               0
          Identity-5             [-1, 197, 384]               0
          Identity-6             [-1, 197, 384]               0
         LayerNorm-7             [-1, 197, 384]             768
            Linear-8            [-1, 197, 1152]         443,520
          Identity-9           [-1, 6, 197, 64]               0
         Identity-10           [-1, 6, 197, 64]               0
          Dropout-11          [-1, 6, 197, 197]               0
           Linear-12             [-1, 197, 384]         147,840
          Dropout-13             [-1, 197, 384]               0
        Attention-14             [-1, 1

In [27]:
# timm.list_models('*resnet*') 이런 식으로 사전학습 가능 모델 검색
resnet_model_list = timm.list_models('*resnet*')
print(resnet_model_list)

['cspresnet50', 'cspresnet50d', 'cspresnet50w', 'eca_resnet33ts', 'ecaresnet26t', 'ecaresnet50d', 'ecaresnet50d_pruned', 'ecaresnet50t', 'ecaresnet101d', 'ecaresnet101d_pruned', 'ecaresnet200d', 'ecaresnet269d', 'ecaresnetlight', 'gcresnet33ts', 'gcresnet50t', 'inception_resnet_v2', 'lambda_resnet26rpt_256', 'lambda_resnet26t', 'lambda_resnet50ts', 'legacy_seresnet18', 'legacy_seresnet34', 'legacy_seresnet50', 'legacy_seresnet101', 'legacy_seresnet152', 'nf_ecaresnet26', 'nf_ecaresnet50', 'nf_ecaresnet101', 'nf_resnet26', 'nf_resnet50', 'nf_resnet101', 'nf_seresnet26', 'nf_seresnet50', 'nf_seresnet101', 'resnet10t', 'resnet14t', 'resnet18', 'resnet18d', 'resnet26', 'resnet26d', 'resnet26t', 'resnet32ts', 'resnet33ts', 'resnet34', 'resnet34d', 'resnet50', 'resnet50_gn', 'resnet50c', 'resnet50d', 'resnet50s', 'resnet50t', 'resnet51q', 'resnet61q', 'resnet101', 'resnet101c', 'resnet101d', 'resnet101s', 'resnet152', 'resnet152c', 'resnet152d', 'resnet152s', 'resnet200', 'resnet200d', 'resn

In [28]:
# 허깅페이스 사전학습 모델 종류 

pretrained_model_list = timm.list_models(pretrained=True)

print(pretrained_model_list)

['bat_resnext26ts.ch_in1k', 'beit_base_patch16_224.in22k_ft_in22k', 'beit_base_patch16_224.in22k_ft_in22k_in1k', 'beit_base_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_224.in22k_ft_in22k', 'beit_large_patch16_224.in22k_ft_in22k_in1k', 'beit_large_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_512.in22k_ft_in22k_in1k', 'beitv2_base_patch16_224.in1k_ft_in1k', 'beitv2_base_patch16_224.in1k_ft_in22k', 'beitv2_base_patch16_224.in1k_ft_in22k_in1k', 'beitv2_large_patch16_224.in1k_ft_in1k', 'beitv2_large_patch16_224.in1k_ft_in22k', 'beitv2_large_patch16_224.in1k_ft_in22k_in1k', 'botnet26t_256.c1_in1k', 'caformer_b36.sail_in1k', 'caformer_b36.sail_in1k_384', 'caformer_b36.sail_in22k', 'caformer_b36.sail_in22k_ft_in1k', 'caformer_b36.sail_in22k_ft_in1k_384', 'caformer_m36.sail_in1k', 'caformer_m36.sail_in1k_384', 'caformer_m36.sail_in22k', 'caformer_m36.sail_in22k_ft_in1k', 'caformer_m36.sail_in22k_ft_in1k_384', 'caformer_s18.sail_in1k', 'caformer_s18.sail_in1k_384', 'caformer_s

In [29]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}

# 데이터셋 사이즈  
# 폴더가 분리되어 있는 경우 len(train_ds)로도 가능
# 폴더가 합쳐져있는 경우 get sampler에서 리턴받은 train_len,val_len으로 가능 
dataset_sizes = {
    "train": len(train_ds),
    "val": len(test_ds)
}

In [30]:
from timm import optim,scheduler

# 최신 논문 구현에서는 Train one epoch, Validate one epoch 함수를 반복하는 형태로 진행
# 제가 한 구현은 Train/validate 모두 하나의 함수에서 진행

# GPU 사용 여부
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_model(model, criterion, num_epochs=config['epochs']):
    
    since = time.time() # 시작 시간
    best_model_wts = copy.deepcopy(model.state_dict()) #
    best_acc = 0.0
    
    '''
    Transformer 학습은 AdamW, CosineLRS를 사용하는 것이 좋다고 알려져 있음
    '''

    #optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # torch에서 제공하는 Opimizer 
    optimizer = timm.optim.AdamW(model.parameters(), lr=1e-3,weight_decay=2e-3) # Hugging Face에서 제공하는 Optimizer
    scheduler = timm.scheduler.cosine_lr.CosineLRScheduler(optimizer,t_initial=num_epochs,warmup_t=20)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)
        
        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate 
            
            running_loss = 0.0
            running_corrects = 0.0

            
            '''
            Mixed Precision 연산은 FP(floating point)16과 FP32를 혼합하여 연산하는 것
            예전에는 amp라는 모듈을 따로 설치해야 했지만, 최근에는 torch에서 제공해줌

            FP16은 FP32에 비해 메모리 사용량이 절반으로 줄어들고, 연산 속도가 빨라짐
            특정 연산은 FP32로 해야하는 경우가 있음 -> Loss/Gradient 계산

            참고자료 : 
            https://velog.io/@twinjuy/Auto-Mixed-Precision%EC%9D%B4%EB%9E%80
            https://computing-jhson.tistory.com/37

            주의할점:
            모델 연산 중 나누기가 있는 경우(ex. 확률화)
            Loss 값이 Nan으로 나오는 경우가 있음, 이러한 경우는 0으로 나눠서 발생하는 문제

            '''
            
            # mixed precision 연산을 위한 scaler
            # scaler는 FP16 연산으로 gradient가 0이 되는 것을 방지하기 위해 scale factor를 곱해주는 역할
            scaler = torch.cuda.amp.GradScaler()

            # 한 Iteration마다 데이터를 불러와서 학습
            for inputs,labels in tqdm(dataloaders[phase]):
                if phase == 'train':
                    # Huggingface에서 제공하는 Mixup 모듈
                    # 하이퍼파라미터는 Transformer 논문들에서 고정적으로 쓰는 값ㅇ르 사용
                    # Mixup은 모듈은 정수 레이블을 사용하더라도 원핫으로 모두 변환해줌
                    mx_fn = Mixup(mixup_alpha=0.8, cutmix_alpha=1.0, num_classes=config['num_classes'])
                    inputs, labels = mx_fn(inputs, labels)
                    
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    # Validation에서는 Mixup을 사용하지 않음
                else:
                    # ImageFolder에서 불러온 레이블은 정수레이블이므로 Mixup결과 레이블에 맞춰 one-hot encoding으로 변환
                    labels = F.one_hot(labels, num_classes=config['num_classes'])
                    inputs = inputs.to(device)
                    labels = labels.to(device)

               
                # 한 iteration 학습 전 gradient 초기화
                # Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문
                optimizer.zero_grad() 
                
                with torch.autocast(device_type = 'cuda',dtype=torch.float16):
                    outputs,_,_ = model(inputs) 
                    _, preds = torch.max(outputs, 1) # max value, index 리턴-> 값은 필요없음
                    
                    loss = criterion(outputs, labels) # loss 계산

                if phase == 'train':
                    # Loss 값 scaling
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                # 한 에폭 학습 후 loss, accuracy 계산

                # loss.item() -> Tensor 변수에서 값만 가져오기
                # inputs.size(0) -> batch size
                # loss*batch_size -> batch size만큼의 loss를 더해줌, 여기에 이후 데이터셋 크기만큼 나누면 iteration당 평균 loss 계산 가능 
                running_loss += loss.item() * inputs.size(0) 
                running_corrects += torch.sum(preds == torch.argmax(labels,dim=-1)) 

            if phase == 'train':
                scheduler.step(epoch) # step at end of epoch
            
            epoch_loss = running_loss / dataset_sizes[phase]
            
            epoch_acc =  running_corrects / dataset_sizes[phase]
            
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model

                model.load_state_dict(best_model_wts)
                torch.save(model,'test_model')                    
        print()

    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [31]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [32]:
criterion = SoftTargetCrossEntropy() # 아니면 mixup에서 label smoothing을 적용해도 됨
criterion = criterion.to(device)

model_ft = train_model(model, criterion) 

model_ft.eval()

Epoch 0/299
----------


  0%|          | 0/1573 [00:02<?, ?it/s]


OSError: Caught OSError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\Capsule\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\user\anaconda3\envs\Capsule\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\user\anaconda3\envs\Capsule\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\user\anaconda3\envs\Capsule\lib\site-packages\torchvision\datasets\folder.py", line 229, in __getitem__
    sample = self.loader(path)
  File "c:\Users\user\anaconda3\envs\Capsule\lib\site-packages\torchvision\datasets\folder.py", line 268, in default_loader
    return pil_loader(path)
  File "c:\Users\user\anaconda3\envs\Capsule\lib\site-packages\torchvisi

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


if __name__ == "__main__":
    test_data = datasets.ImageFolder(root= 'kproduct_test', transform = T.Compose([ T.Resize((224,224)), T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]))

    test_loader = DataLoader(test_data, batch_size=32, sampler=None ,shuffle=False, num_workers=4) 

    model = torch.load('test_model').to(device)
    torchsummary.summary(model, (3, 224, 224))
    model.eval()

    criterion = SoftTargetCrossEntropy()
    criterion = criterion.to(device)

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)

            # Loss 계산
            loss = criterion(outputs, labels)

            # ACC 계산
            _, predicted = torch.max(outputs.data, 1)
            total = labels.size(0)
            correct = (predicted == labels).sum().item()
            accuracy = correct / total
        
                
    print("Loss: {:.4f} Acc: {:.4f}".format(loss, accuracy))
